In [1]:
import shelve
from time import sleep
import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
import requests
from vk_api.execute import VkFunction
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import datetime
import numpy as np

In [2]:
start_comm_id = 21000000
comm_N = 25000

In [3]:
vk_func_getMembQuan = VkFunction(args=('x'), code='''
    var memb = [];
    var i = 0;
    while (i < 25) 
    {
        memb.push(API.groups.getMembers({"group_id": i + %(x)s})["count"]);
        i = i + 1;
    };
    return memb;
''')

vk_func_getMembIDList = VkFunction(args=('x', 'y'), code='''
    var offset = %(y)s;
    var IDList = [];
    var i = 0;
    while (i < 25)
    {
        IDList.push(API.groups.getMembers({"group_id": %(x)s, "offset" : offset + i * 1000})["items"]);
        i = i + 1;
    }
    return IDList;
''')

In [4]:
vk_session = vk_api.VkApi(token='d082c8543db0fb23abd5ba5e8e8ef60dee208a5e5373068b4e7ebdb37d46ec61a31474d57f9e99481a403') 
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)

Login is required to auth


In [5]:
vk_session.RPS_DELAY = 0.05
vk = vk_session.get_api()
#vk.RPS_DELAY = 0.05
print('--start--')

MQData = []
for comm_id in range(start_comm_id, start_comm_id + comm_N, 25):
    MQData.append(vk_func_getMembQuan(vk, comm_id))
    print(comm_id - start_comm_id)

--start--
0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3200
3225
3250
3275
3300
3325
3350
3375
3400
3425
3450
3475
3500
3525
3550
3575
3600
3625
3650
3675
3700
3725
3750
3775
3800
3825
3850
3875
3900
3925
3950
3975
4000
4025
4050
4075
4100
4125
4150
4175
4200
4225
4250
4275
4300
4325
4350
4375
4400
4425
4450
4475
4500
4525
4550
4575
4600
4625
4650
4675
4700
4725
4750
4775
4800
4825
4850
4875
4900
4925
4950
4975
5000
5025
5050
5075
5100
5125
5150


In [7]:
MQData = [y for x in MQData for y in x]
print(MQData)

[13, 1, 1, 1, 2, 1, 16, 2, 1, 2306, 1, 142, 1, 1, 0, 0, None, 120, 170, 0, 81, 4, 0, 1, 1, 7, 3276, None, None, 16, 30, 12, 20, 1, None, 0, 2, 1, 34, 0, 11, 4, 196, 49, 1, 106, 469, 2, 1558, 3, 409, 31, 24, 2, 2, 4, 1, 0, 13, 4, 463, None, 0, 2, 302, None, 36, 61, None, 50, 5, 1, 239, 7, 94, 3, 6, 3, 1, 1, 865, 251, 2, 45, 8, 63, 0, None, 3, 2, 28, 58, 6, 0, 9, 1, 1, 1, 1, 1, 2, 15, 46, 2, 0, 1177, 184, 6, 16, 559, 14, 4, 0, 219, 5, 4, 0, 5, 3, 19, 203, 119, 39, None, 1, 10, None, 5, None, 0, 285, 38, 5, 7, 9, 0, 1, 1, 33, 0, 15, 1, None, 1, 94, None, 2, 20, None, 16, 4, 0, 1, None, 8, 223, 6095, 0, 14, 0, 1, 3, 0, 322, 2, None, 2, 1, 0, 1, 1, 2, 0, 0, 0, 3, 0, None, 11, 12, 3, 12, 39, 18, 6, 24, 11, 3, 0, 84, None, None, 8630, 0, 4, 3, 0, 13, 1, 4, 0, 11, None, 2, 0, 11, 107, 119, 122, 2, 40, 2, 2, 265, 1, 690, 0, 1, None, 300, 34, 0, 2, 0, 32, 2027, 2, 0, 1, 0, 112, 494, 2, 45, 411, 8, 4, 673, 1, 0, 2, 344, 301, 0, 1, 0, 6, 29, None, 2, 13, 1, 8, 2, 39, 1, 17, 179, 8, 146, 1, None, 1

In [8]:
MQDict = {}
for i in range(comm_N):
    if (MQData[i] is not None) and (MQData[i] > 1e4):
        MQDict[start_comm_id + i] = MQData[i]
print(MQDict)

{20902097: 21148, 20903061: 19337, 20903968: 34133, 20904034: 16915, 20906682: 10643, 20907498: 16286, 20907640: 29871, 20909759: 10002, 20909815: 13605}


In [9]:
IDDict = {}
for id, quan in MQDict.items():
    IDDict[id] = []
    for offset in range(0, quan, 25000):
        IDDict[id].append(vk_func_getMembIDList(vk, x=id, y=offset))
        
for id, ilist in IDDict.items():
    IDDict[id] = [z for x in IDDict[id] for y in x for z in y]

In [10]:
np.save(f'{start_comm_id}to{start_comm_id + comm_id - 1}list.npy', IDDict) 

In [11]:
for key, val in IDDict.items():
    print(f"{key}: {val[0:10000]}")

20902097: [7328, 9571, 24324, 53257, 63612, 68363, 81492, 86666, 116140, 143764, 143980, 175530, 176117, 188323, 191544, 192217, 199791, 201591, 202216, 211326, 242500, 282764, 284512, 285404, 285745, 289592, 291689, 300642, 302468, 327554, 345498, 346761, 356085, 359853, 433597, 442138, 450671, 455695, 460592, 473149, 492676, 524269, 571142, 573505, 606149, 635114, 664484, 730523, 792432, 808906, 942740, 1043920, 1051583, 1058359, 1179436, 1199757, 1232277, 1235140, 1241313, 1333184, 1347303, 1355985, 1397827, 1450527, 1459444, 1467200, 1475812, 1512403, 1548706, 1572164, 1583788, 1667014, 1729143, 1753872, 1755211, 1818207, 1832943, 1876796, 1892716, 1914334, 1962058, 1975325, 2019636, 2024317, 2046938, 2048974, 2062359, 2069948, 2141295, 2158299, 2164056, 2242017, 2247268, 2302734, 2316079, 2358801, 2395685, 2423721, 2505563, 2550506, 2557611, 2585684, 2605026, 2606103, 2641284, 2647976, 2666555, 2699319, 2715132, 2876080, 2905456, 2951621, 2961628, 2972407, 2982321, 2988815, 299239